In [ ]:
import pandas as pd
# from ydata_profiling import ProfileReport

In [ ]:
df = pd.read_csv('csv/uspevaemost.csv', encoding='cp1251')

In [ ]:
df = df[df['Статус'] == 'Отчисленный']

df = df[df['Квалификация'] != 'Слушатель программ довузовской подготовки']

In [ ]:
df.drop_duplicates(inplace=True)

df = df[~(df['Оценка'] == 'неудовлетворительно') | ~(df.duplicated(subset=df.columns.difference(['Оценка']))) ]

df['Оценка'] = df['Оценка'].map({
    'отлично': 5,
    'хорошо': 4,
    'удовлетворительно': 3,
    'зачтено': 2, 
    'незачет': 1 
})

df_bez_D = df.drop_duplicates(subset=df.columns.difference(['Оценка']), keep=False)

df_s_D = df.merge(df_bez_D, indicator=True, how='outer').loc[lambda x: x['_merge'] == 'left_only'].drop(columns=['_merge']).drop_duplicates()

In [ ]:
df = df_s_D
df[df['Студент'] == '7A74794B6C504F644431394D5A652F517757433847413D3D'].sort_values(by=['Учебный год', 'Семестр'])

In [ ]:
max_indices = df_s_D.groupby(['Студент', 'Группа', 'Дисциплина', 'Семестр', 'Учебный год', 'Специальность', 'Форма обучения', 'Квалификация', 'Статус'])['Оценка'].idxmax(skipna=True)
df_max = df_s_D.loc[max_indices]


min_indices = df_s_D.groupby(['Студент', 'Группа', 'Дисциплина', 'Семестр', 'Учебный год', 'Специальность', 'Форма обучения', 'Квалификация', 'Статус'])['Оценка'].idxmin(skipna=True)
df_min = df_s_D.loc[min_indices]

In [ ]:
df_min = df_min.groupby(['Студент', 'Группа', 'Семестр', 'Учебный год', 'Специальность', 'Форма обучения', 'Квалификация', 'Статус'], as_index=True)['Оценка'].value_counts().unstack(fill_value=0)
df_max = df_max.groupby(['Студент', 'Группа', 'Семестр', 'Учебный год', 'Специальность', 'Форма обучения', 'Квалификация', 'Статус'], as_index=True)['Оценка'].value_counts().unstack(fill_value=0)
df_bez_D = df_bez_D.groupby(['Студент', 'Группа', 'Семестр', 'Учебный год', 'Специальность', 'Форма обучения', 'Квалификация', 'Статус'], as_index=True)['Оценка'].value_counts().unstack(fill_value=0)

In [ ]:

df = pd.merge(df_bez_D, df_max, on=['Студент','Группа','Семестр','Учебный год','Специальность','Форма обучения','Квалификация','Статус'], how='outer')
df = pd.merge(df, df_min, on=['Студент','Группа','Семестр','Учебный год','Специальность','Форма обучения','Квалификация','Статус'], how='outer')

df.fillna(0, inplace=True)


In [ ]:
df.rename(columns={'1_x': 'незачет', '2_x': 'зачет', '3_x': 'удовлетворительно', '4_x': 'хорошо', '5_x': 'отлично', '2_y': 'зачет испр', '3_y': 'удовлетворительно испр', '4_y': 'хорошо испр', '5_y': 'отлично испр', '1_y': 'незачет до испр', 2: 'зачет до испр', 3: 'удовлетворительно до испр', 4: 'хорошо до испр'}, inplace=True)

In [ ]:

phrases = ['незачет', 'зачет', 'удовлетворительно', 'хорошо',
         'отлично', 'зачет испр', 'удовлетворительно испр',
         'хорошо испр', 'отлично испр', 'незачет до испр',
         'зачет до испр', 'удовлетворительно до испр', 'хорошо до испр']

df = df.sort_values(by=['Студент', 'Семестр'])
for phrase in phrases:
    df[f'Накоп {phrase}'] = df.groupby('Студент')[f'{phrase}'].cumsum()


In [ ]:
df['незачет'] = df['незачет'].apply(lambda x: 1 if x >= 1 else 0)

df['отчислен'] = df.groupby('Студент')['незачет'].transform('max')

In [ ]:
df = df[df['незачет'] < 1]
df = df[df['Накоп незачет'] < 1]



In [ ]:
df.to_csv('tempData.csv', index=True)
df = pd.read_csv('tempData.csv')

df['Квалификация'] = df['Квалификация'].map({
    'Аспирант': 3,
    'Магистр': 2,
    'Специалист': 1,
    'Бакалавр': 0,
})

df['Форма обучения'] = df['Форма обучения'].map({
    'Заочная': 2,
    'Очная': 0,
   'Очно-заочная': 1
})

In [ ]:

df.drop('Статус', axis=1, inplace=True)
df.drop('Студент', axis=1, inplace=True)
df.drop('Группа', axis=1, inplace=True)
df.drop('Специальность', axis=1, inplace=True)
df.drop('Учебный год', axis=1, inplace=True)
# df.drop('незачет', axis=1, inplace=True)
# df.drop('Накоп незачет', axis=1, inplace=True)


In [ ]:
df.to_csv('csv/dataset.csv', index=False)




In [ ]:
df.head()

In [ ]:
profile = ProfileReport(df, title="Profiling Report")
profile.to_file("your_reportXD167.html")

df.to_csv('csv/uspevaemost1000.csv', index=False)